In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv('data\AbandonoEmpleados.csv', sep =';', index_col='id', na_values='#N/D')
df

,edad,abandono,viajes,departamento,distancia_casa,educacion,carrera,empleados,satisfaccion_entorno,sexo,...,satisfaccion_companeros,horas_quincena,nivel_acciones,anos_experiencia,num_formaciones_ult_ano,conciliacion,anos_compania,anos_en_puesto,anos_desde_ult_promocion,anos_con_manager_actual
id,,,,,,,,,,,,,,,,,,,,,
1,41,Yes,Travel_Rarely,Sales,1,Universitaria,Life Sciences,1,Media,3.0,...,Baja,80,0,8,0,NaN,6,NaN,0,5
2,49,No,Travel_Frequently,Research & Development,8,Secundaria,Life Sciences,1,Alta,2.0,...,Muy_Alta,80,1,10,3,NaN,10,NaN,1,7
4,37,Yes,Travel_Rarely,Research & Development,2,Secundaria,Other,1,Muy_Alta,2.0,...,Media,80,0,7,3,NaN,0,2.0,0,0
5,33,No,Travel_Frequently,Research & Development,3,Universitaria,Life Sciences,1,Muy_Alta,3.0,...,Alta,80,0,8,3,NaN,8,3.0,3,0
7,27,No,Travel_Rarely,Research & Development,2,Universitaria,Medical,1,Baja,3.0,...,Muy_Alta,80,1,6,3,NaN,2,NaN,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2061,36,No,Travel_Frequently,Research & Development,23,Master,Medical,1,Alta,4.0,...,Alta,80,1,17,3,NaN,5,4.0,0,3
2062,39,No,Travel_Rarely,Research & Development,6,Secundaria,Medical,1,Muy_Alta,2.0,...,Baja,80,1,9,5,NaN,7,NaN,1,7
2064,27,No,Travel_Rarely,Research & Development,4,Master,Life Sciences,1,Media,4.0,...,Media,80,1,6,0,NaN,6,NaN,0,3


# BUSINESS ANALYTICS

In [ ]:
df.info()

## Null Analysis

In [ ]:
miss = df.isna().sum().sort_values(ascending=False)
miss

In [ ]:

miss_percentage = miss / len(df)
filtered = miss_percentage[miss_percentage != 0]
filtered

In [ ]:
plt.figure(figsize=(6,3))
sns.barplot(x=filtered.values, y=filtered.index).set(title='Percentage of missing values', ylabel='Variables')


#### Conclusions
- anos_en_puesto, conciliacion -> delete variables
- sexo,educacion,satisfaccion_trabajo,implicacion -> imputed after EDA

### Elimination of variables with 30% or more missing data

In [ ]:
df.drop(columns=['anos_en_puesto','conciliacion'], inplace=True)
df.info()

In [ ]:
obj =df.select_dtypes('O')
obj.shape[1]

## EDA - Categorical VariablesData

In [ ]:
from math import ceil

def eda_categorical_chart(cat):
    n_rows_subplot = ceil(cat.shape[1]/2)
    g,ax = plt.subplots(nrows=n_rows_subplot, ncols= 2, figsize= (16, n_rows_subplot*6))
    ax = ax.flat
    
    for my_index, my_column in enumerate(cat):
        cat[my_column].value_counts().plot.barh(ax=ax[my_index])
        ax[my_index].set_title(my_column,fontsize=12,fontweight='bold')
        ax[my_index].tick_params(labelsize=12)
        

In [ ]:
eda_categorical_chart(df.select_dtypes('O'))

### Conclutions
* mayor_edad has a single value -> delete variable

* pending inputations:
    * educacion -> Universitaria 
    * satisfaccion_trabajo -> Alta
    * implicacion -> Alta

In [ ]:
df.drop(columns = 'mayor_edad', inplace=True)
df['educacion'] = df['educacion'].fillna('Universitaria')
df['satisfaccion_trabajo'] = df['satisfaccion_trabajo'].fillna('Alta')
df['implicacion'] = df['implicacion'].fillna('Alta')

## EDA - Numerical Variables

In [ ]:
df.select_dtypes('number').describe()

In [ ]:
def my_describe(num):
    des = num.describe().T
    des['median'] = num.median()
    #sorting columns
    des = des.iloc[:,[0,1,8,2,3,4,5,6,7]]
    return(des)

In [ ]:
my_describe(df.select_dtypes('number'))

### Conclutions
* empleados has a single value -> delete
* sexo has 4 values -> delete/ we don't know why it has 4 values.
* horas_quincena has a single value -> delete.

In [ ]:
df.drop(columns=['empleados', 'sexo', 'horas_quincena'], inplace=True)


## INSIGHTS

### What is the abandonment rate?

In [ ]:
abandonment_rate = df.abandono.value_counts(normalize=True)*100
abandonment_rate

In [ ]:
plt.figure(figsize=(3,2))
sns.barplot(x=abandonment_rate.index, y=abandonment_rate.values)

### Is there a employee profile that leaves the company?

In [ ]:
# Transforming abandono values to numerical
df['abandono'] = df.abandono.map({'No':0, 'Yes':1})

In [ ]:
temp = df.groupby('educacion').abandono.mean().sort_values(ascending=False)*100
plt.figure(figsize=(5, 2))
ax = temp.plot.bar()
ax.set_xticklabels(temp.index, rotation = 0)
plt.show()

In [ ]:
temp = df.groupby('estado_civil').abandono.mean().sort_values(ascending=False)*100
plt.figure(figsize=(5, 2))
ax = temp.plot.bar()
ax.set_xticklabels(temp.index, rotation = 0)
plt.show()

In [ ]:
temp = df.groupby('horas_extra').abandono.mean().sort_values(ascending=False)*100
plt.figure(figsize=(5, 3))
ax = temp.plot.bar()
ax.set_xticklabels(temp.index, rotation = 0)
plt.show()

In [ ]:
temp = df.groupby('puesto').abandono.mean().sort_values(ascending=False)*100
plt.figure(figsize=(15, 6))
ax = temp.plot.bar()
ax.set_xticklabels(temp.index, rotation = 15)
plt.show()

In [ ]:
temp = df.groupby('abandono').salario_mes.mean().sort_values(ascending=False)
plt.figure(figsize=(5, 3))
ax = temp.plot.bar()
ax.set_xticklabels(temp.index, rotation = 0)
plt.show()

### Profile
* low educational level - Primaria
* Single
* Works in sales
* Works overtime

### Wich is the economic impact of this problem?

In [ ]:
# Creating a new variable anual_salary
df['salario_anual'] = df.salario_mes.transform(lambda x: x*12)
df[['salario_mes', 'salario_anual']]

In [ ]:
# Calculte the economic impact if the employee leave the company
conditions = [(df['salario_anual'] <= 30000),
              (df['salario_anual'] > 30000) & (df['salario_anual'] < 50000),
              (df['salario_anual'] > 50000) & (df['salario_anual'] <= 75000),
              (df['salario_anual'] > 75000)]

results = [df.salario_anual * 0.161, df.salario_anual * 0.197,\
           df.salario_anual * 0.204, df.salario_anual * 0.21]

df['impacto_abandono'] = np.select(conditions, results, default=-999)
df.head()

### How much has this problem cost us in the last year?

In [ ]:
coste_total = df.loc[df.abandono == 1].impacto_abandono.sum()
coste_total

### How much does it cost us that employees aren't motivated?

In [ ]:
df.loc[(df.abandono == 1) & (df.implicacion == 'Baja')].impacto_abandono.sum()

### How much could we save by building employee loyalty?

In [ ]:
print(f"Reducir un 10% la fuga de empleados nos ahorraría {int(coste_total * 0.1)}$ cada año.")

print(f"Reducir un 20% la fuga de empleados nos ahorraría {int(coste_total * 0.2)}$ cada año.")

print(f"Reducir un 30% la fuga de empleados nos ahorraría {int(coste_total * 0.3)}$ cada año.")